# Основные Улучшения
Анализ Галлюцинаций:

1. Добавлен метод analyze_hallucinations в классе GPT для логирования потенциальных галлюцинаций в сгенерированных ответах.
Поддержка Русского Языка:

2. Интегрирована возможность загрузки модели для работы с русским языком (если такая доступна) и обработки ответов на русском.
Фильтрация Запросов:

3. Реализованы базовые проверки в методе validate_answer для предотвращения неподобающих или вредоносных запросов.

# Проект: Нейро-сотрудник как Консультант по обучению

# Установка необходимых библиотек

Описание: Эта команда устанавливает библиотеки, необходимые для работы с OpenAI API, создания интерфейса с Gradio и работы с векторными базами данных.

In [ ]:
!pip install openai gradio tiktoken langchain langchain-openai langchain-community chromadb

In [ ]:
models = [
    {
        "doc": "https://docs.google.com/document/d/1f7Gfv2PZYACD1PGzlonfBZsWI9Pf9ZOFI_xYp3DRxw0/edit",
        "prompt": '''Ты Консультант по обучению. Я помогу вам находить информацию и ресурсы по обучению.
                    Сообщите мне, как я могу помочь вам!''',
        "name": "Нейро-сотрудник консультант",
        "query": "Как найти ресурсы по обучению?"
    },
]

# Импорт библиотек и настройка окружения

Описание: Здесь мы импортируем библиотеки и устанавливаем ключ API для доступа к OpenAI. getpass позволяет безопасно вводить ключ.

In [ ]:
import logging
import getpass
import os
import requests
import re
import gradio as gr
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from openai import OpenAI
import tiktoken
import chromadb

In [ ]:
# Настройка логирования
logging.basicConfig(
    filename='app.log',
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")


Введите OpenAI API Key:··········


# Определение класса GPT

Описание: Класс GPT инициализирует модель и содержит методы для загрузки документов и получения ответов.

# Реализация методов класса
def create_embedding(self, data):

Описание: Эти методы позволяют создавать эмбеддинги из документов и получать ответы на запросы пользователей.

In [ ]:
class GPT():
    def __init__(self, model="gpt-3.5-turbo"):
        self.model = model
        self.search_index = None
        self.client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
        logging.info("Инициализация GPT с моделью: %s", model)

    def load_search_indexes(self, url):
        match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
        if match_ is None:
            logging.error('Неверный Google Docs URL: %s', url)
            raise ValueError('Неверный Google Docs URL')
        doc_id = match_.group(1)
        try:
            response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
            response.raise_for_status()
            text = response.text
            return self.create_embedding(text)
        except requests.exceptions.RequestException as e:
            logging.error('Ошибка при загрузке документа: %s', e)
            raise

    def num_tokens_from_string(self, string):
        encoding = tiktoken.encoding_for_model(self.model)
        num_tokens = len(encoding.encode(string))
        logging.debug('Количество токенов в строке: %d', num_tokens)
        return num_tokens

    def create_embedding(self, data):
        source_chunks = []
        splitter = CharacterTextSplitter(separator="\n", chunk_size=1024, chunk_overlap=0)
        for chunk in splitter.split_text(data):
            source_chunks.append(Document(page_content=chunk, metadata={}))
        count_token = self.num_tokens_from_string(' '.join([x.page_content for x in source_chunks]))
        logging.info('Количество токенов в документе: %d', count_token)
        try:
            self.search_index = Chroma.from_documents(source_chunks, OpenAIEmbeddings())
            logging.info('Данные из документа загружены в векторную базу данных')
        except Exception as e:
            logging.error('Ошибка при создании векторной базы данных: %s', e)
            raise

    def answer_index(self, system, topic, temp=1):
        if not self.search_index:
            logging.warning('Модель необходимо обучить!')
            return ''
        docs = self.search_index.similarity_search(topic, k=5)
        message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'Отрывок документа №{i+1}:\n' + doc.page_content for i, doc in enumerate(docs)]))
        messages = [
            {"role": "system", "content": system + f"{message_content}"},
            {"role": "user", "content": topic}
        ]
        try:
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=temp
            )
            logging.info('Ответ получен от модели')
            return completion.choices[0].message.content
        except Exception as e:
            logging.error('Ошибка при получении ответа от модели: %s', e)
            return 'Ошибка при получении ответа.'

    def analyze_hallucinations(self, answer, query):
        # Пример анализа галлюцинаций
        if "неизвестно" in answer.lower():
            logging.warning(f"Галлюцинация обнаружена: {answer}")
            return True
        return False

# Kласс NeuroEmployee.

Описание: Этот класс будет использоваться для предоставления ответов на запросы пользователей в зависимости от их профессиональной роли.

validate_answer, context_check, и другие.

Описание: Каждый метод отвечает за определенную логику, например, проверка валидности ответов или анализ контекста.

In [ ]:
class NeuroEmployee:
    def __init__(self, profession):
        self.profession = profession
        self.system_prompt = self.create_system_prompt()
        self.client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        self.knowledge_base = self.initialize_knowledge_base()

    def create_system_prompt(self):
        if self.profession == 'consultant':
            return "Ты — консультант. Твоя задача — давать советы и рекомендации на основе предоставленных данных."
        elif self.profession == 'analyst':
            return "Ты — аналитик. Твоя задача — анализировать данные и предоставлять выводы."
        else:
            return "Ты — нейро-сотрудник. Используй свои знания для помощи пользователю."

    def initialize_knowledge_base(self):
        return []  # Заглушка для вашей логики базы знаний

    def create_embedding(self, text):
        response = self.client.embeddings.create(
            input=text,
            model="text-embedding-ada-002"
        )
        return response.data[0].embedding

    def load_documents(self, documents):
        for doc in documents:
            embedding = self.create_embedding(doc)
            self.knowledge_base.append({'document': doc, 'embedding': embedding})

    def validate_answer(self, answer, query):
        # Пример запроса к API для проверки фактов
        try:
            response = requests.get(f"https://api.example.com/validate?query={query}&answer={answer}")
            response.raise_for_status()  # Проверка на наличие ошибок HTTP
            return response.json().get('is_valid', False)
        except requests.exceptions.RequestException as e:
            logging.error(f"Ошибка при проверке ответа: {e}")
            return False  # Если произошла ошибка, считать ответ невалидным

    def context_check(self, answer, query):
        # Проверка семантической связи с использованием эмбеддингов
        answer_embedding = self.create_embedding(answer)
        query_embedding = self.create_embedding(query)

        # Здесь можно использовать косинусное сходство или другую метрику для оценки схожести
        similarity = self.calculate_similarity(answer_embedding, query_embedding)
        return similarity > 0.7  # Порог схожести, можно настроить

    def calculate_similarity(self, embedding1, embedding2):
        # Пример расчета косинусного сходства
        import numpy as np
        dot_product = np.dot(embedding1, embedding2)
        norm_a = np.linalg.norm(embedding1)
        norm_b = np.linalg.norm(embedding2)
        return dot_product / (norm_a * norm_b)

    def answer_query(self, query):
        full_prompt = f"{self.system_prompt}\n\nЗапрос: {query}"
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": full_prompt}]
        )
        answer = response.choices[0].message.content

        # Проверка валидности ответа
        if not self.validate_answer(answer, query):
            logging.warning(f"Сгенерированный ответ невалиден: {answer}")
            return self.retry_answer(full_prompt)

        # Проверка контекста
        if not self.context_check(answer, query):
            logging.warning(f"Сгенерированный ответ не соответствует контексту: {answer}")
            return self.retry_answer(full_prompt)

        # Анализ галлюцинаций
        if self.analyze_hallucinations(answer, query):
            return "Ответ может быть невалидным. Пожалуйста, уточните запрос."

        return answer

    def retry_answer(self, prompt):
        # Попробовать снова с тем же запросом
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content

    def update_knowledge_base(self, new_documents):
        self.load_documents(new_documents)

# Создание интерфейса с Gradio

Описание: Здесь мы создаем интерфейс для выбора модели и ввода запросов.

Тестирование и запуск:
demo.launch()
Описание: Эта команда запускает веб-интерфейс, который позволяет пользователям взаимодействовать с моделью.

In [ ]:
# Создание объектов
gpt = GPT("gpt-3.5-turbo")
neuro_employee = NeuroEmployee(profession='consultant')

# Gradio интерфейс
blocks = gr.Blocks()

with blocks as demo:
    subject = gr.Dropdown([(elem["name"], index) for index, elem in enumerate(models)], label="Данные")
    name = gr.Label(show_label=False)
    prompt = gr.Textbox(label="Промт", interactive=True)
    link = gr.HTML()
    query = gr.Textbox(label="Запрос к LLM", interactive=True)

    def onchange(dropdown):
        if dropdown is None or dropdown == "":
            return ["", "", "", ""]
        return [
            models[dropdown]['name'],
            models[dropdown]['prompt'].strip().replace('\t', ' ').replace('  ', ' '),
            models[dropdown]['query'],
            f"<a target='_blank' href = '{models[dropdown]['doc']}'>Документ для обучения</a>"
        ]

    subject.change(onchange, inputs=[subject], outputs=[name, prompt, query, link])

    with gr.Row():
        train_btn = gr.Button("Обучить модель")
        request_btn = gr.Button("Запрос к модели")

    def train(dropdown):
        if dropdown is None or dropdown == "":
            logging.error('Выбор модели не был сделан.')
            return 'Ошибка: выбор модели не был сделан.'

        logging.debug('Выбранный индекс модели: %s', dropdown)
        if dropdown < 0 or dropdown >= len(models):
            logging.error('Индекс модели вне диапазона: %s', dropdown)
            return 'Ошибка: индекс модели вне диапазона.'

        try:
            gpt.load_search_indexes(models[dropdown]['doc'])
            logging.info('Обучение модели завершено')
            return "Обучение завершено."
        except Exception as e:
            logging.error('Ошибка при обучении модели: %s', e)
            return f'Ошибка при обучении модели: {e}'

    def predict(p, q):
        result = neuro_employee.answer_query(q)  # Использование NeuroEmployee
        return [result, "Логирование завершено."]

    with gr.Row():
        response = gr.Textbox(label="Ответ LLM")
        log = gr.Textbox(label="Логирование")

    train_btn.click(train, [subject], log)
    request_btn.click(predict, [prompt, query], [response, log])

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fc5f88a9af74b69192.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
